## **Lab 2: KNN**

In [129]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from typing import Optional, Literal, Tuple


In [130]:
torch.manual_seed(42)
device_default = torch.device("cuda:0" if torch.cuda.is_available()
                              else ("mps" if torch.backends.mps.is_available() else "cpu")
                              )

### 1. Dataset


In [131]:
insurance_df = pd.read_csv("datasets/insurance.csv").dropna()
insurance_df.head()


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [132]:
insurance_df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [133]:
fashion_df = pd.read_csv("datasets/fashion-mnist_test.csv").dropna()
fashion_df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [134]:
fashion_df.describe()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000
mean,4.500000,0.000400,0.010300,0.052100,0.077000,0.208600,0.349200,0.826700,2.321200,5.457800,...,34.320800,23.071900,16.432000,17.870600,22.860000,17.790200,8.353500,2.541600,0.629500,0.06560
std,2.872425,0.024493,0.525187,2.494315,2.208882,4.669183,5.657849,8.591731,15.031508,23.359019,...,57.888679,49.049749,42.159665,44.140552,51.706601,45.128107,28.765769,16.417363,7.462533,1.93403
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,4.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
75%,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,55.000000,6.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000
max,9.000000,2.000000,45.000000,218.000000,185.000000,227.000000,223.000000,247.000000,218.000000,244.000000,...,254.000000,252.000000,255.000000,255.000000,255.000000,255.000000,240.000000,225.000000,205.000000,107.00000


In [135]:
def regression_metrics(y_true: torch.Tensor, y_pred: torch.Tensor) -> Tuple[float, float, float]:
    y_true = y_true.to(y_pred.device)
    mse = torch.mean((y_true - y_pred) ** 2).item()
    mae = torch.mean(torch.abs(y_true - y_pred)).item()
    rmse = mse ** 0.5
    return mse, mae, rmse


### 2. Data Processing

In [136]:
insurance_df["sex"] = insurance_df["sex"].str.lower().map({"male": 0, "female": 1})
insurance_df["smoker"] = insurance_df["smoker"].str.lower().map({"no": 0, "yes": 1})
insurance_df["region"] = insurance_df["region"].str.lower().map({"southeast": 0, "southwest": 1,  "northwest": 2, "northeast": 3})

insurance_df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,1,27.900,0,1,1,16884.92400
1,18,0,33.770,1,0,0,1725.55230
2,28,0,33.000,3,0,0,4449.46200
3,33,0,22.705,0,0,2,21984.47061
4,32,0,28.880,0,0,2,3866.85520


### 3. Model

In [137]:
class KNN:
    def __init__(
        self,
        k: int = 5,
        task: Literal["classification", "regression"] = "classification",
        weights: Literal["uniform", "distance"] = "uniform",
        standardize: bool = True,
        num_classes: Optional[int] = None,
        device: Optional[torch.device] = None,
        metric: Literal["euclidean", "manhattan", "cosine"] = "euclidean",
        eps: float = 1e-12,
    ) -> None:
        # Validate parameters
        assert k >= 1, "k must be >= 1"
        assert task in ("classification", "regression")
        assert weights in ("uniform", "distance")
        assert metric in ("euclidean", "manhattan", "cosine")

        self.k = int(k)
        self.task = task
        self.weights = weights
        self.standardize = standardize
        self.num_classes = num_classes
        self.device = device if device is not None else device_default
        self.metric = metric
        self.eps = float(eps)

        # Internal attributes
        self.X_train: Optional[torch.Tensor] = None
        self.y_train: Optional[torch.Tensor] = None
        self.mean: Optional[torch.Tensor] = None
        self.std: Optional[torch.Tensor] = None
        self.classes_: Optional[torch.Tensor] = None

    def _standardize_fit(self, X: torch.Tensor) -> torch.Tensor:
        if not self.standardize:
            return X
        self.mean = X.mean(dim=0, keepdim=True)
        self.std = X.std(dim=0, unbiased=False, keepdim=True).clamp_min(1e-12)
        return (X - self.mean) / self.std

    def _standardize_transform(self, X: torch.Tensor) -> torch.Tensor:
        """Standardize new data using training mean/std."""
        if not self.standardize or self.mean is None or self.std is None:
            return X
        return (X - self.mean) / self.std

    def _compute_distances(self, A: torch.Tensor, B: torch.Tensor) -> torch.Tensor:
        """Compute pairwise distances between two sets of vectors."""
        if self.metric == "euclidean":
            return torch.cdist(A, B, p=2)
        elif self.metric == "manhattan":
            return torch.cdist(A, B, p=1)
        else:  # cosine distance
            A_norm = A / (A.norm(dim=1, keepdim=True).clamp_min(self.eps))
            B_norm = B / (B.norm(dim=1, keepdim=True).clamp_min(self.eps))
            similarity = A_norm @ B_norm.T
            distance = 1.0 - similarity
            return distance

    def fit(self, X: torch.Tensor, y: torch.Tensor) -> "KNN":
        """Fit model with training data."""
        X = X.to(self.device, dtype=torch.float32)

        if self.task == "classification":
            y = y.to(self.device)
            classes, y_encoded = torch.unique(y, sorted=True, return_inverse=True)
            self.classes_ = classes.to(self.device)
            self.y_train = y_encoded.to(self.device, dtype=torch.long).contiguous()
            if self.num_classes is None:
                self.num_classes = int(self.classes_.numel())
        else:  # regression
            self.y_train = y.to(self.device, dtype=torch.float32).contiguous()

        X_standardized = self._standardize_fit(X)
        self.X_train = X_standardized.contiguous()
        return self

    @torch.no_grad()
    def predict(self, X: torch.Tensor) -> torch.Tensor:
        """Predict labels or values for new samples."""
        assert self.X_train is not None and self.y_train is not None, "Call fit first"
        X = X.to(self.device, dtype=torch.float32)
        X_standardized = self._standardize_transform(X)

        # Compute distances and select k nearest neighbors
        distances = self._compute_distances(X_standardized, self.X_train)
        neighbor_distances, neighbor_indices = torch.topk(
            distances, k=self.k, dim=1, largest=False
        )

        if self.task == "classification":
            neighbor_labels = self.y_train[neighbor_indices].long()

            if self.weights == "uniform":
                weights = torch.ones_like(neighbor_distances, dtype=torch.float32)
            else:  # distance-based weights
                weights = 1.0 / (neighbor_distances + self.eps)

            # Handle exact matches (distance = 0)
            zero_mask = (neighbor_distances <= self.eps)
            rows_with_zero = zero_mask.any(dim=1)
            if rows_with_zero.any():
                weights[rows_with_zero] = torch.where(
                    zero_mask[rows_with_zero],
                    torch.ones_like(weights[rows_with_zero]),
                    torch.zeros_like(weights[rows_with_zero])
                )

            # Aggregate scores for each class
            scores = torch.zeros(
                (X_standardized.size(0), self.num_classes),
                device=self.device,
                dtype=torch.float32,
            )
            scores.scatter_add_(1, neighbor_labels, weights)

            pred_encoded = scores.argmax(dim=1).to(torch.long)
            return self.classes_[pred_encoded] if self.classes_ is not None else pred_encoded

        else:  # regression
            neighbor_values = self.y_train[neighbor_indices]
            if self.weights == "uniform":
                weights = torch.ones_like(neighbor_distances, dtype=torch.float32)
            else:
                weights = 1.0 / (neighbor_distances + self.eps)
            predictions = (weights * neighbor_values).sum(dim=1) / weights.sum(dim=1)
            return predictions

### 4. Model Training

In [138]:
features = insurance_df.drop(columns=["charges"]).values
targets = insurance_df["charges"].values

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

knn_model = KNN(k=5, task="regression", weights="distance", standardize=True, device=device_default, metric="euclidean")

knn_model.fit(X_train_tensor, y_train_tensor)
y_pred_tensor = knn_model.predict(X_test_tensor)

mse, mae, rmse = regression_metrics(y_test_tensor, y_pred_tensor)
print(f"MSE: {mse:.2f}, MAE: {mae:.2f}, RMSE: {rmse:.2f}")

MSE: 26512832.00, MAE: 2998.35, RMSE: 5149.06


In [139]:
pca = PCA(n_components=10)
fashion_test_df = fashion_df
X_fashion_train = torch.from_numpy(np.array(pca.fit_transform(fashion_df.drop('label', axis=1)), dtype='float32')).to(device_default)
X_fashion_test = torch.from_numpy(np.array(pca.transform(fashion_test_df.drop('label', axis=1)), dtype='float32')).to(device_default)
y_fashion_train = torch.from_numpy(fashion_df.to_numpy(dtype=np.float32)).to(device_default)
y_fashion_test = torch.from_numpy(fashion_test_df.to_numpy(dtype=np.float32)).to(device_default)

knn_clf = KNN(k=10, task="classification", weights="distance", standardize=False, device=device_default)

knn_clf.fit(X_fashion_train, y_fashion_train)

y_pred_clf = knn_clf.predict(X_fashion_test)

torch.sum(y_pred_clf == y_fashion_test)/len(y_pred_clf)

y_pred_clf

RuntimeError: Index tensor must have the same number of dimensions as self tensor